In [ ]:
// Kotlin notebook
import jline.*
import jline.lang.*
import jline.lang.layered.*
import jline.lang.nodes.*
import jline.lang.processes.*
import jline.lang.constant.*
import jline.solvers.ln.*
import jline.solvers.mva.*
import jline.util.matrix.*

In [ ]:
println("Example of layered model with a function task (FaaS)")

val model = LayeredNetwork("faas_test_example")

In [ ]:
// Definition of processors, tasks and entries
val P1 = Processor(model, "P1", GlobalConstants.MaxInt, SchedStrategy.INF)
val T1 = Task(model, "T1", 1, SchedStrategy.REF).on(P1)
val E1 = Entry(model, "E1").on(T1)

val P2 = Processor(model, "P2", 4, SchedStrategy.FCFS)
// val T2 = Task(model, "T2", 1, SchedStrategy.FCFS).on(P2)
// Function task with capacity, scheduling strategy, think time, setup time, and delay-off time
val T2 = FunctionTask(model, "F2", 6, SchedStrategy.FCFS).on(P2).setThinkTime(Exp.fitMean(8.0))
T2.setSetupTime(Exp(1.0))      // Cold start time
T2.setDelayOffTime(Exp(2.0))   // Time before function instance is removed

val E2 = Entry(model, "E2").on(T2)

// val T3 = Task(model, "T3", 1, SchedStrategy.FCFS).on(P2)
// val E3 = Entry(model, "E3").on(T3)

In [ ]:
// Definition of activities
val A1 = Activity(model, "A1", Exp(1.0)).on(T1).boundTo(E1).synchCall(E2, 1)
val A2 = Activity(model, "A2", Exp(3.0)).on(T2).boundTo(E2).repliesTo(E2)
// val A3 = Activity(model, "A3", Exp(5.0)).on(T3).boundTo(E3).repliesTo(E3)

In [ ]:
// Define a helper function for solver selection (as in original MATLAB)
fun myFunction(model: LayeredNetwork): NetworkSolver {
    // Helper function to select appropriate solver for the model
    // This could implement logic to choose between MVA, MAM, etc.
    // For now, use MVA as default
    val options = MVA.defaultOptions()
    options.verbose = 0
    return MVA(model, options)
}

// Solve with LN solver using custom function
val lnoptions = LN.defaultOptions()
lnoptions.verbose = 0
lnoptions.seed = 2300

try {
    val solver = LN(model, ::myFunction, lnoptions)
    val avgTable = solver.avgTable
    avgTable.print()
} catch (e: Exception) {
    println("Error: ${e.message}")
}

This FaaS (Function-as-a-Service) layered model demonstrates:

1. **Function-as-a-Service Architecture**:
   - **Client layer**: T1 (reference task) representing users/applications
   - **Function layer**: T2 (FunctionTask) representing serverless functions
   - **Processor constraints**: Limited resources for function execution

2. **FunctionTask Characteristics**:
   - **Capacity**: 6 concurrent function instances
   - **Think time**: 8-unit mean (idle time between invocations)
   - **Setup time**: 1-unit exponential (cold start penalty)
   - **Delay-off time**: 2-unit exponential (keep-alive duration)

3. **Cold Start Behavior**:
   - **Setup time**: Models function initialization overhead
   - **Delay-off time**: Models how long functions stay "warm"
   - **Performance impact**: Cold starts add latency to function calls

4. **Resource Management**:
   - **P1**: Infinite capacity (client-side, no resource constraints)
   - **P2**: 4 processors (limited compute resources for functions)
   - **Function instances**: Up to 6 concurrent executions

5. **Service Pattern**:
   - **Synchronous calls**: Client waits for function response
   - **Service time**: 3-unit exponential for function execution
   - **Request rate**: Determined by client think time and system capacity

6. **Solver Integration**:
   - **Custom function**: `myFunction()` selects appropriate solver
   - **LN**: Layered Network solver with pluggable algorithms
   - **MVA backend**: Mean Value Analysis for performance calculation

The model represents:
- **Serverless computing**: AWS Lambda, Azure Functions, Google Cloud Functions
- **Microservices**: Function-based service architectures
- **Edge computing**: Distributed function execution
- **API gateways**: Function invocation through service endpoints

Key insights:
- **Cold start impact**: Setup time significantly affects response time for infrequent functions
- **Keep-alive optimization**: Delay-off time balances resource usage vs. performance
- **Capacity planning**: Function instances vs. processor capacity affects scalability

**FaaS Performance Factors**:
- **Request frequency**: High frequency reduces cold start impact
- **Resource allocation**: More processors support more concurrent functions
- **Function complexity**: Longer service times increase resource contention

**Design Trade-offs**: The model captures the fundamental tension in serverless computing between resource efficiency (quick scale-down) and performance (avoiding cold starts).